In [0]:
# Airflow Integration Parameters

try:
    # Create widgets for Airflow parameters
    dbutils.widgets.text("batch_id", "manual_run", "Batch ID from Airflow")
    dbutils.widgets.text("execution_date", "", "Execution Date from Airflow") 
    dbutils.widgets.text("force_refresh", "false", "Force data refresh")
    dbutils.widgets.text("quality_threshold", "0.8", "Data quality threshold")
    dbutils.widgets.text("dag_run_id", "", "DAG Run ID")
    
    # Get parameter values
    batch_id = dbutils.widgets.get("batch_id")
    execution_date = dbutils.widgets.get("execution_date")
    force_refresh = dbutils.widgets.get("force_refresh").lower() == "true"
    quality_threshold = float(dbutils.widgets.get("quality_threshold"))
    dag_run_id = dbutils.widgets.get("dag_run_id")
    
    print(f"🎯 Airflow Parameters:")
    print(f"   Batch ID: {batch_id}")
    print(f"   Execution Date: {execution_date}")
    print(f"   Force Refresh: {force_refresh}")
    print(f"   Quality Threshold: {quality_threshold}")
    print(f"   DAG Run ID: {dag_run_id}")
    
except Exception as e:
    print(f"⚠️ Widget creation failed (normal in some contexts): {e}")
    # Fallback values for manual runs
    batch_id = "manual_run"
    execution_date = ""
    force_refresh = False
    quality_threshold = 0.8
    dag_run_id = ""

In [0]:
# Event Hub News Consumer with FinBERT Processing
print("📰 Event Hub News Consumer with FinBERT")
print("=" * 60)

# Store Python's built-in round before imports
import builtins
python_round = builtins.round

# Import required modules
import json
import time
import threading
from datetime import datetime, timezone
from azure.eventhub import EventHubConsumerClient, TransportType
from pyspark.sql.functions import *
from pyspark.sql.types import *
import traceback

# Enhanced schema setup with ALL STRING types to prevent ANY conflicts
print("🏗️ Setting up database schemas with ALL STRING types...")

def create_schemas_and_tables():
    """Create catalog, schema, and tables for news data - ALL STRING TYPES"""
    
    try:
        print("🔍 Checking existing schemas...")
        
        # Get current catalog name
        current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
        current_database = spark.sql("SELECT current_database()").collect()[0][0]
        
        print(f"📁 Current catalog: {current_catalog}")
        print(f"📁 Current database: {current_database}")
        
        # Show existing schemas with improved column access
        try:
            schemas_df = spark.sql("SHOW SCHEMAS")
            schema_columns = schemas_df.columns
            print(f"📋 Schema columns available: {schema_columns}")
            
            if 'namespace' in schema_columns:
                existing_schemas = [row['namespace'] for row in schemas_df.collect()]
            elif 'databaseName' in schema_columns:
                existing_schemas = [row['databaseName'] for row in schemas_df.collect()]
            elif 'schemaName' in schema_columns:
                existing_schemas = [row['schemaName'] for row in schemas_df.collect()]
            else:
                existing_schemas = [row[0] for row in schemas_df.collect()]
            
            print(f"📋 Existing schemas: {existing_schemas}")
            
        except Exception as schema_list_error:
            print(f"⚠️ Could not list schemas: {schema_list_error}")
            existing_schemas = []
        
        # Create silver schema if it doesn't exist
        if 'silver' not in existing_schemas:
            print("🔧 Creating silver schema...")
            spark.sql(f"CREATE SCHEMA IF NOT EXISTS {current_catalog}.silver")
            print("✅ Silver schema created")
        else:
            print("✅ Silver schema already exists")
            
        # Create bronze schema if it doesn't exist (for backup verification)
        if 'bronze' not in existing_schemas:
            print("🔧 Creating bronze schema...")
            spark.sql(f"CREATE SCHEMA IF NOT EXISTS {current_catalog}.bronze")
            print("✅ Bronze schema created")
        else:
            print("✅ Bronze schema already exists")
        
        # Create COMPLETELY NEW TABLE with ALL STRING types to avoid ALL conflicts
        new_news_table = f"{current_catalog}.silver.news_data_consumer"
        
        # Drop existing table if it exists
        try:
            spark.sql(f"DROP TABLE IF EXISTS {new_news_table}")
            print("🗑️ Dropped any existing consumer table")
        except:
            pass
        
        # Create silver news_data table with ALL STRING types
        news_table_ddl = f"""
        CREATE TABLE {new_news_table} (
            title STRING,
            content STRING,
            source STRING,
            published_at STRING,
            url STRING,
            original_sentiment_score STRING,     -- STRING to avoid precision conflicts
            original_sentiment_category STRING,
            finbert_label STRING,
            finbert_score STRING,               -- STRING to avoid precision conflicts
            finbert_confidence STRING,          -- STRING to avoid precision conflicts
            finbert_negative STRING,            -- STRING to avoid precision conflicts
            finbert_neutral STRING,             -- STRING to avoid precision conflicts
            finbert_positive STRING,            -- STRING to avoid precision conflicts
            processing_method STRING,
            processing_time_seconds STRING,     -- STRING to avoid precision conflicts
            text_length STRING,                 -- STRING to avoid INT/LONG conflicts
            consumer_timestamp STRING,
            partition_id STRING,
            processing_mode STRING,
            silver_processing_time STRING,
            ingestion_time STRING,
            processed_date STRING,
            ingestion_batch STRING,
            layer STRING,
            ingestion_source STRING
        )
        USING DELTA
        """
        
        spark.sql(news_table_ddl)
        print(f"✅ Table {new_news_table} created with ALL STRING types")
        
        # Verify tables exist
        tables = spark.sql(f"SHOW TABLES IN {current_catalog}.silver").collect()
        table_names = [row.tableName for row in tables]
        print(f"📋 Available tables in silver schema: {table_names}")
        
        return current_catalog, new_news_table
        
    except Exception as e:
        print(f"❌ Error creating schemas: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Create schemas and get catalog name
current_catalog, news_table_name = create_schemas_and_tables()

print("\n" + "=" * 60)
print("📰 Running News Consumer with FinBERT")
print("=" * 60)

# Configuration
try:
    eh_connection_string = dbutils.secrets.get(scope="stock-project", key="event-hub-connection-string")
    if "EntityPath=" not in eh_connection_string:
        eh_connection_string = f"{eh_connection_string};EntityPath=stock-data-hub"

    storage_account_key = dbutils.secrets.get(scope="stock-project", key="storage-account-key")
    storage_account_name = "dlsstocksentiment2025"
    container_name = "data"

    spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

    adls_base_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"
    silver_news_path = f"{adls_base_path}/silver/news_data_consumer"

    print("✅ Configuration loaded")
    
except Exception as config_error:
    print(f"❌ Configuration error: {config_error}")
    raise

# Load FinBERT with improved error handling
finbert_ready = False
try:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import torch
    import numpy as np
    
    print("🤖 Loading FinBERT model...")
    device = torch.device('cpu')
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    model.eval()
    finbert_ready = True
    print("✅ FinBERT model ready")
    
except Exception as e:
    print(f"⚠️ FinBERT failed, using fallback: {e}")
    finbert_ready = False

def apply_finbert_sentiment(text):
    """Apply FinBERT sentiment analysis with robust fallback - RETURN ALL STRINGS"""
    if not finbert_ready:
        # Enhanced fallback sentiment analysis
        text_lower = text.lower()
        positive_words = ['gain', 'rise', 'profit', 'bullish', 'up', 'growth', 'strong', 'beat', 'outperform', 'surge']
        negative_words = ['loss', 'fall', 'bearish', 'down', 'decline', 'weak', 'miss', 'underperform', 'drop', 'crash']
        
        positive_count = sum(1 for word in positive_words if word in text_lower)
        negative_count = sum(1 for word in negative_words if word in text_lower)
        
        if positive_count > negative_count:
            return {
                'finbert_label': 'positive', 
                'finbert_score': '0.7', 
                'finbert_confidence': '0.8', 
                'finbert_negative': '0.1',
                'finbert_neutral': '0.2',
                'finbert_positive': '0.7',
                'method': 'fallback'
            }
        elif negative_count > positive_count:
            return {
                'finbert_label': 'negative', 
                'finbert_score': '-0.7', 
                'finbert_confidence': '0.8',
                'finbert_negative': '0.7',
                'finbert_neutral': '0.2',
                'finbert_positive': '0.1',
                'method': 'fallback'
            }
        else:
            return {
                'finbert_label': 'neutral', 
                'finbert_score': '0.0', 
                'finbert_confidence': '0.7',
                'finbert_negative': '0.33',
                'finbert_neutral': '0.34',
                'finbert_positive': '0.33',
                'method': 'fallback'
            }
    
    try:
        # Limit text length and clean it
        clean_text = text.strip()[:512]
        if not clean_text:
            clean_text = "No content"
            
        inputs = tokenizer(clean_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        predictions = predictions.numpy()[0]
        labels = ['negative', 'neutral', 'positive']
        
        max_idx = np.argmax(predictions)
        predicted_label = labels[max_idx]
        confidence = float(predictions[max_idx])
        
        # Calculate score: positive for positive sentiment, negative for negative
        if predicted_label == 'positive':
            score = confidence
        elif predicted_label == 'negative':
            score = -confidence
        else:
            score = 0.0
        
        # RETURN ALL VALUES AS STRINGS
        return {
            'finbert_label': str(predicted_label),
            'finbert_score': str(python_round(score, 4)),
            'finbert_confidence': str(python_round(confidence, 4)),
            'finbert_negative': str(python_round(float(predictions[0]), 4)),
            'finbert_neutral': str(python_round(float(predictions[1]), 4)),
            'finbert_positive': str(python_round(float(predictions[2]), 4)),
            'method': 'finbert'
        }
    except Exception as e:
        print(f"⚠️ FinBERT processing error: {e}")
        return {
            'finbert_label': 'neutral', 
            'finbert_score': '0.0', 
            'finbert_confidence': '0.5',
            'finbert_negative': '0.33',
            'finbert_neutral': '0.34',
            'finbert_positive': '0.33',
            'method': 'error'
        }

def safe_save_to_catalog(df, table_name, mode="append"):
    """Safely save to Unity Catalog with fallback"""
    try:
        (df.write
         .format("delta")
         .mode(mode)
         .option("mergeSchema", "false")  # No schema merging to avoid conflicts
         .saveAsTable(table_name))
        return True
    except Exception as e:
        print(f"⚠️ Unity Catalog save failed for {table_name}: {e}")
        return False

def safe_parse_json(message_body):
    """Safely parse JSON with better error handling"""
    try:
        # Handle empty or whitespace-only messages
        if not message_body or not message_body.strip():
            print(f"⚠️ Empty message body")
            return None
            
        # Handle non-JSON messages (like "Test connection message")
        message_body = message_body.strip()
        if not message_body.startswith('{') and not message_body.startswith('['):
            print(f"⚠️ Non-JSON message: {message_body[:50]}...")
            return None
            
        # Try to parse JSON
        data = json.loads(message_body)
        return data
        
    except json.JSONDecodeError as je:
        print(f"⚠️ JSON decode error: {je} - Raw message: {message_body[:100]}...")
        return None
    except Exception as e:
        print(f"⚠️ Message parsing error: {e}")
        return None

def debug_print_message_structure(event, index):
    """Debug function to print message structure"""
    try:
        print(f"\n🔍 DEBUG - Message {index} structure:")
        print(f"   Keys: {list(event.keys())}")
        print(f"   data_type: {event.get('data_type', 'MISSING')}")
        print(f"   title: {event.get('title', 'MISSING')[:50]}...")
        
        # Print all key-value pairs for first few messages
        if index <= 2:
            for key, value in event.items():
                if key in ['title', 'content']:
                    print(f"   {key}: {str(value)[:50]}... (type: {type(value).__name__})")
                else:
                    print(f"   {key}: {value} (type: {type(value).__name__})")
                
    except Exception as e:
        print(f"⚠️ Debug print error: {e}")

def news_finbert_consume_and_process():
    """News consumer with FinBERT processing - ALL STRING TYPES"""
    
    print("\n🔄 Starting news data consumption and FinBERT processing...")
    
    # Step 1: Consume messages with timeout and validation
    messages = []
    stop_flag = threading.Event()
    
    def consumer_thread():
        try:
            client = EventHubConsumerClient.from_connection_string(
                eh_connection_string,
                consumer_group="$Default",
                transport_type=TransportType.AmqpOverWebsocket
            )
            
            def message_handler(partition_context, event):
                if stop_flag.is_set() or len(messages) >= 12:
                    return
                
                if event and hasattr(event, 'body_as_str'):
                    try:
                        body = event.body_as_str(encoding='UTF-8')
                        
                        # Use safe JSON parsing
                        data = safe_parse_json(body)
                        if data is None:
                            return  # Skip invalid messages
                        
                        # Filter for news data only
                        if data.get('data_type') == 'news_sentiment':
                            data['consumer_timestamp'] = datetime.now(timezone.utc).isoformat()
                            data['partition_id'] = partition_context.partition_id
                            messages.append(data)
                            title = data.get('title', 'No title')[:30]
                            print(f"📰 {len(messages)}: NEWS - {title}...")
                            
                            if len(messages) >= 12:
                                stop_flag.set()
                        else:
                            # Skip non-news messages silently
                            pass
                            
                    except Exception as me:
                        print(f"⚠️ Message processing error: {me}")
            
            with client:
                client.receive(
                    on_event=message_handler,
                    starting_position="-1",
                    max_wait_time=12
                )
        except Exception as e:
            print(f"⚠️ Consumer error: {e}")
            stop_flag.set()
    
    # Start consumer thread
    thread = threading.Thread(target=consumer_thread, daemon=True)
    thread.start()
    
    # Wait with extended timeout
    start_time = time.time()
    timeout_seconds = 25
    while thread.is_alive() and time.time() - start_time < timeout_seconds and len(messages) < 12:
        time.sleep(0.5)
    
    stop_flag.set()
    thread.join(timeout=3)
    
    print(f"✅ Consumed {len(messages)} news messages")
    
    if not messages:
        print("⚠️ No news messages consumed - Event Hub might be empty or only contains stock data")
        return
    
    # Debug: Print first message structure
    if messages:
        debug_print_message_structure(messages[0], 1)
    
    # Step 2: Process news data with FinBERT - ALL STRING TYPES
    news_events = [m for m in messages if m.get('data_type') == 'news_sentiment']
    
    print(f"\n🤖 Processing {len(news_events)} news events with FinBERT:")
    
    # Process news with enhanced FinBERT analysis - ALL STRING OUTPUT
    processed_news = []
    for i, event in enumerate(news_events):
        try:
            print(f"\n🔍 Processing news {i+1}/{len(news_events)}")
            
            title = str(event.get('title', '') or '')
            content = str(event.get('content', '') or '')
            
            # Enhanced text preparation
            if not title and not content:
                title = f"Article {i+1}"
                content = "No content available"
            
            full_text = f"{title}. {content}".strip()[:1000]  # Limit length
            print(f"   Title: {title[:50]}...")
            
            # Apply FinBERT with timing
            start_time = time.time()
            result = apply_finbert_sentiment(full_text)
            processing_time = time.time() - start_time
            
            # ALL VALUES AS STRINGS to eliminate any type conflicts
            processed_news.append({
                'title': str(title[:500]) if title else "",
                'content': str(content[:2000]) if content else "",
                'source': str(event.get('source', 'unknown')),
                'published_at': str(event.get('published_at', datetime.now(timezone.utc).isoformat())),
                'url': str(event.get('url', '')[:500]),
                'original_sentiment_score': str(event.get('sentiment_score', 0.0)),  # STRING
                'original_sentiment_category': str(event.get('sentiment_category', 'neutral')),
                'finbert_label': str(result['finbert_label']),
                'finbert_score': str(result['finbert_score']),                      # STRING
                'finbert_confidence': str(result['finbert_confidence']),            # STRING
                'finbert_negative': str(result.get('finbert_negative', '0.33')),    # STRING
                'finbert_neutral': str(result.get('finbert_neutral', '0.34')),      # STRING
                'finbert_positive': str(result.get('finbert_positive', '0.33')),    # STRING
                'processing_method': str(result['method']),
                'processing_time_seconds': str(python_round(processing_time, 4)),   # STRING
                'text_length': str(len(full_text)),                                 # STRING
                'consumer_timestamp': str(event.get('consumer_timestamp', '')),
                'partition_id': str(event.get('partition_id', '')),
                'processing_mode': str('streaming_news_finbert'),
                'silver_processing_time': str(datetime.now(timezone.utc).isoformat())
            })
            
            print(f"✅ {i+1}/{len(news_events)}: {title[:30]}... → {result['finbert_label']} ({result['finbert_score']}) [{result['method']}]")
            
        except Exception as e:
            print(f"❌ News processing error {i+1}: {e}")
            traceback.print_exc()
            continue
    
    # Step 3: Save news data with ALL STRING metadata
    batch_id = f"news_consumer_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    save_success = False
    
    if processed_news:
        try:
            print(f"\n💾 Saving {len(processed_news)} FinBERT processed news articles...")
            
            # Add metadata as strings
            current_time_str = datetime.now(timezone.utc).isoformat()
            current_date_str = datetime.now().strftime('%Y-%m-%d')
            
            for record in processed_news:
                record["ingestion_time"] = str(current_time_str)
                record["processed_date"] = str(current_date_str)
                record["ingestion_batch"] = str(batch_id)
                record["layer"] = str("silver")
                record["ingestion_source"] = str("news_consumer")
            
            news_df = spark.createDataFrame(processed_news)
            
            # Debug: Show schema before saving
            print("📋 News DataFrame schema:")
            news_df.printSchema()
            
            # Save to ADLS (primary) - NO SCHEMA MERGING
            (news_df.write
             .format("delta")
             .mode("append")
             .option("mergeSchema", "false")
             .save(silver_news_path))
            
            print("✅ Saved FinBERT news to Silver layer (ADLS)")
            save_success = True
            
            # Try Unity Catalog (secondary)
            if current_catalog and news_table_name and safe_save_to_catalog(news_df, news_table_name):
                print("✅ Also saved news to Unity Catalog")
            else:
                print("⚠️ Unity Catalog news save skipped")
            
        except Exception as e:
            print(f"❌ News save error: {e}")
            traceback.print_exc()
    
    # Step 4: Verification and reporting
    print(f"\n🔍 Verification:")
    try:
        if save_success:
            try:
                # Verify ADLS storage
                news_files = dbutils.fs.ls(silver_news_path)
                print(f"✅ Silver layer news files: {len(news_files)}")
                
                # Verify data integrity
                news_verify = spark.read.format("delta").load(silver_news_path)
                total_news = news_verify.count()
                recent_news = news_verify.filter(col("ingestion_source") == "news_consumer").count()
                print(f"✅ Verified: {recent_news} new FinBERT records ({total_news} total)")
                
                # Show sample of what was saved
                print(f"\n📊 Sample FinBERT Results:")
                (news_verify
                 .filter(col("ingestion_source") == "news_consumer")
                 .select("title", "finbert_label", "finbert_score", "finbert_confidence", "processing_method")
                 .orderBy(col("silver_processing_time").desc())
                 .limit(5)
                 .show(truncate=False))
                
                # Show sentiment distribution
                print(f"\n📈 Sentiment Distribution:")
                (news_verify
                 .filter(col("ingestion_source") == "news_consumer")
                 .groupBy("finbert_label")
                 .agg(count("*").alias("count"))
                 .orderBy("finbert_label")
                 .show())
                
            except Exception as ve:
                print(f"⚠️ News verification error: {ve}")
        
    except Exception as e:
        print(f"⚠️ Verification error: {e}")
        traceback.print_exc()
    
    # Final summary
    print(f"\n📋 Final Summary:")
    print(f"✅ Consumed: {len(messages)} news messages")
    print(f"✅ Processed: {len(processed_news)} news articles with FinBERT")
    print(f"✅ FinBERT model: {'Real model' if finbert_ready else 'Enhanced fallback'}")
    print(f"✅ ADLS save: {save_success}")
    print(f"📁 Batch ID: {batch_id}")
    print(f"📋 New Table: {news_table_name}")

# Execute consumer with top-level error handling
try:
    print("🚀 Starting News FinBERT Consumer Script")
    print(f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    news_finbert_consume_and_process()
    
except Exception as e:
    print(f"❌ News FinBERT consumer failed: {e}")
    print("📋 Full error traceback:")
    traceback.print_exc()

print(f"\n⏰ COMPLETED: {datetime.now().strftime('%H:%M:%S')}")


📰 Event Hub News Consumer with FinBERT
🏗️ Setting up database schemas with ALL STRING types...
🔍 Checking existing schemas...
📁 Current catalog: databricks_stock_sentiment_canada
📁 Current database: default
📋 Schema columns available: ['databaseName']
📋 Existing schemas: ['bronze', 'default', 'information_schema', 'silver']
✅ Silver schema already exists
✅ Bronze schema already exists
🗑️ Dropped any existing consumer table
✅ Table databricks_stock_sentiment_canada.silver.news_data_consumer created with ALL STRING types
📋 Available tables in silver schema: ['news_data_consumer', 'stock_data_consumer']

📰 Running News Consumer with FinBERT
✅ Configuration loaded
🤖 Loading FinBERT model...


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

✅ FinBERT model ready
🚀 Starting News FinBERT Consumer Script
⏰ 2025-07-21 22:13:46

🔄 Starting news data consumption and FinBERT processing...
📰 1: NEWS - Stock market today: Live updat...
📰 2: NEWS - Is It Time To Consider Buying ...
📰 3: NEWS - Your Photography Skills Are Al...
📰 4: NEWS - ASX Penny Stocks With Market C...
📰 5: NEWS - Is spinal cord stimulation saf...
📰 6: NEWS - ⚡ Nepal’s EV Revolution Surges...
📰 7: NEWS - High Growth Tech Stocks in Aus...
📰 8: NEWS - Stocks only go up…until they d...
✅ Consumed 8 news messages

🔍 DEBUG - Message 1 structure:
   Keys: ['title', 'content', 'source', 'published_at', 'url', 'sentiment_score', 'data_type', 'title_length', 'content_length', 'api_fetch_time', 'sentiment_category', 'event_timestamp', 'event_source', 'partition_key', 'processing_mode', 'consumer_timestamp', 'partition_id']
   data_type: news_sentiment
   title: Stock market today: Live updates...
   title: Stock market today: Live updates... (type: str)
   content: U.S. s

In [0]:
%python
import json
from datetime import datetime

# Airflow Integration - Success/Failure Reporting

try:
    # If we reach here, notebook executed successfully
    success_result = {
        "status": "SUCCESS",
        "message": "Notebook execution completed successfully",
        "batch_id": batch_id,
        "execution_timestamp": datetime.now().isoformat(),
        "records_processed": locals().get('total_records_processed', 0),  # Update based on your variables
        "data_quality_score": locals().get('data_quality_score', 1.0)     # Update based on your variables
    }
    
    print(f"✅ Notebook Success:")
    print(json.dumps(success_result, indent=2))
    
    # Exit with success status for Airflow
    dbutils.notebook.exit(success_result)
    
except Exception as e:
    # If any error occurs, report failure
    failure_result = {
        "status": "FAILED", 
        "message": f"Notebook execution failed: {str(e)}",
        "batch_id": batch_id,
        "execution_timestamp": datetime.now().isoformat(),
        "error_type": type(e).__name__
    }
    
    print(f"❌ Notebook Failure:")
    print(json.dumps(failure_result, indent=2))
    
    # Exit with failure status for Airflow
    dbutils.notebook.exit(failure_result)